In [8]:
import pandas as pd
import numpy as np

# 步骤1: 读取 CSV 文件
df = pd.read_csv('F:/AAA/GNN_Gaussian/data/PEMS04/distance.csv')

# 步骤2: 提取 cost 列
cost_series = df['cost']

# 步骤3: 将 cost 列的值转换为列
cost_values = cost_series.to_numpy().reshape(1, -1)

# 步骤4: 重复这些列，直到总共有 16992 行
num_rows_needed = 16992
num_repeats = num_rows_needed // len(cost_values) + (1 if num_rows_needed % len(cost_values) != 0 else 0)
repeated_data = np.tile(cost_values, (num_repeats, 1))

# 截取前 16992 行
repeated_data = repeated_data[:16992]

# 将重复的数据转换为 DataFrame
repeated_df = pd.DataFrame(repeated_data)


# 步骤5: 将结果保存到新的 CSV 文件中
repeated_df.to_csv('F:/AAA/GNN_Gaussian/data/PEMS04/edge_attr.csv', index=False)

print("File saved as 'edge_attr.csv'")

File saved as 'edge_attr.csv'


In [ ]:
import numpy as np
import pandas as pd
from statsmodels.tsa.seasonal import STL

# 假设你的三维数据已经加载到名为 data 的 NumPy 数组中
# data 的形状为 (16992, 307, 3)，其中 16992 是时间点数量，307 是变量数量，3 是每个变量的观测值数量
data = np.load('F:/AAA/STGODE/data/PEMS04/PEMS04.npz')
# 首先，我们需要将数据转换为一个 DataFrame，其中每个变量形成一个列
num_timestamps, num_variables, num_obs = data.shape
data_2d = data.reshape(num_timestamps, num_variables * num_obs)

# 创建一个 DataFrame
df = pd.DataFrame(data_2d, columns=[f'var_{i}_obs_{j}' for i in range(num_variables) for j in range(num_obs)])

# 初始化一个字典来保存每个变量的 STL 分解结果
stl_results = {}

# 对每个变量进行 STL 分解
for var in range(num_variables):
    # 获取当前变量的时间序列数据
    time_series = df[f'var_{var}_obs_0']  # 假设我们只对每个变量的第一个观测值进行分解

    # 创建 STL 对象并拟合数据
    stl = STL(time_series, seasonal=13)  # 这里的 seasonal 参数取决于数据的季节性周期，可能需要调整
    res = stl.fit()

    # 保存 STL 分解结果
    stl_results[f'var_{var}'] = res

# 现在，stl_results 字典包含了每个变量的 STL 分解结果
# 你可以访问每个变量的趋势、季节性和残差组件
# 例如，获取第一个变量的趋势组件：
trend_component = stl_results[f'var_0'].trend

# 如果你需要，可以将这些组件保存为 CSV 文件
for var, res in stl_results.items():
    trend = res.trend
    season = res.seasonal
    residual = res.resid

    # 保存每个变量的趋势、季节性和残差为 CSV 文件
    trend.to_csv(f'{var}_trend.csv', header=False)
    season.to_csv(f'{var}_seasonal.csv', header=False)
    residual.to_csv(f'{var}_residual.csv', header=False)

In [10]:
import pandas as pd

# 步骤1: 读取 CSV 文件
df = pd.read_csv('F:/AAA/GNN_Gaussian/data/PEMS04/distance.csv')

# 步骤2: 获取第一列的数据
first_column = df.iloc[:, 1]

# 步骤3: 将第一列的数据用逗号连接起来
# 使用 str.join 方法将 Series 中的元素连接成一个字符串，每个元素之间用逗号分隔
comma_separated = ','.join(first_column.dropna().astype(str))

# 步骤4: 打印结果
print(comma_separated)

5,154,263,56,96,42,58,95,72,271,68,107,130,129,167,298,209,146,170,173,117,0,92,243,62,80,97,28,57,55,223,143,269,290,110,121,299,293,148,150,152,70,255,131,132,133,18,43,118,207,169,127,208,297,168,166,226,26,94,219,217,31,215,111,116,36,78,20,138,284,114,245,48,206,144,172,24,35,115,86,214,27,216,218,76,238,50,52,75,171,7,1,46,270,32,10,247,249,225,260,259,103,302,104,71,88,268,240,9,239,23,22,49,258,157,158,286,102,285,15,8,300,34,161,125,235,163,236,250,122,252,69,39,234,82,274,175,177,213,179,33,181,183,184,185,254,188,141,278,289,190,192,194,196,198,180,135,54,231,66,59,232,14,89,228,205,244,100,303,136,305,139,151,149,295,291,294,77,109,292,147,29,222,79,90,2,81,204,229,4,246,11,87,74,165,241,108,137,123,37,84,101,221,220,201,211,210,262,306,83,199,197,195,193,191,189,280,279,140,187,186,296,126,182,248,25,178,142,176,174,113,124,253,30,67,164,119,120,61,19,162,51,160,159,64,287,267,288,283,281,282,156,38,65,230,47,265,264,53,45,12,41,272,106,17,63,202,16,40,105,233,277,275,264,